In [2]:
import librosa
import os
import numpy as np
import scipy.signal
import random
import math
from scipy.io import loadmat
from IPython.display import Audio
import os
import random,librosa

In [6]:
#pos_x and pos_y are integers from 1 to 9; snr_ratio should be from 0 to 1
def generate_examples(speech_dir,noise_dir,srir_dir,pos_x,pos_y,snr_ratio):
    #load in mono speech
    src_audio, sr = librosa.load(speech_dir, sr=44100, mono=True)
    src_audio /= src_audio.max()
    
    #convolve speech with srir
    grid_x=pos_x
    grid_y=pos_y
    ch_out_list = []
    sh_names = ["W", "X", "Y", "Z"]
    for sh_str in sh_names:
        #ch_ir_path = os.path.join(srir_dir, sh_str,
        #                      "{}x{:02d}y{:02d}.wav".format(sh_str, grid_x, grid_y))
        ch_ir, sr = librosa.load(srir_dir, sr=44100)
        
        ch_ir_len = ch_ir.shape[0]
        src_len = src_audio.shape[0]
    
        if ch_ir_len > src_len:
            pad_len = ch_ir_len - src_len
            src_audio = np.pad(src_audio, (0, pad_len), mode='constant')
        elif ch_ir_len < src_len:
            pad_len = src_len - ch_ir_len
            ch_ir = np.pad(ch_ir, (0, pad_len), mode='constant')
        
        ch_out = scipy.signal.fftconvolve(src_audio, ch_ir, mode='full')[:src_len]
        ch_out_list.append(ch_out)

    src_bformat = np.array(ch_out_list)
    
    #load in bformat noise
    noise_data = None
    while noise_data is None or noise_data.shape[1] < src_bformat.shape[1]:
        noise_data, sr = librosa.load(noise_dir, sr=44100, mono=False)
    
    #align bformat noise and bformat speech
    clip_len = src_bformat.shape[1]
    start_idx = np.random.randint(0, noise_data.shape[1] - clip_len)
    noise_data = noise_data[:,start_idx:start_idx + clip_len]
    
    #designate snr and scale
    snr = 10 * np.log10(np.mean(src_bformat[0,:] ** 2) / np.mean(noise_data[0,:] ** 2))
    snr_target = snr_ratio * 40.0 - 20.0
    alpha = 10.0**((snr_target - snr) / 20.0)#scaling factor
    src_bformat *= alpha
    
    #combine the noise+speech
    mix_bformat = src_bformat + noise_data
    
    return mix_bformat
    
    

In [7]:
speech_dir='./vctk-p225/p225_003.wav'
noise_dir = './ambiencelondonstreet.wav'
srir_dir='./isophonics/greathall/Z/Zx00y03.wav'
pos_x=0
pos_y=0
snr_ratio=0.1
mix_bformat=generate_examples(speech_dir,noise_dir,srir_dir,pos_x,pos_y,snr_ratio)

/Users/lilyh/anaconda3/lib/python3.6/site-packages/scipy/signal/signaltools.py:375: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  complex_result = (np.issubdtype(in1.dtype, complex) or
/Users/lilyh/anaconda3/lib/python3.6/site-packages/scipy/signal/signaltools.py:376: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  np.issubdtype(in2.dtype, complex))


In [9]:
def rotate_90(audio):
    return scipy.signal.hilbert(audio).imag
sr=44100
# Mix to stereo according to https://en.wikipedia.org/wiki/Ambisonic_UHJ_format#UHJ_encoding_and_decoding_equations
# S = 0.9396926*W + 0.1855740*X
# D = j(-0.3420201*W + 0.5098604*X) + 0.6554516*Y
# Left = (S + D)/2.0
# Right = (S - D)/2.0
S = 0.9396926 * mix_bformat[0] + 0.1855740 * mix_bformat[1]
D = rotate_90(-0.3420201 * mix_bformat[0] + 0.5098604 * mix_bformat[1]) + 0.6554516 * mix_bformat[2]
L = (S + D)/2.0
R = (S - D)/2.0
mix_mono = S
mix_stereo = np.stack([L,R])

import IPython.display as ipd

ipd.Audio(mix_stereo,rate=sr)